# <center> <b> <h1> Gowalla, Stalker pattern recognition </h1></b></center>

<h4 id="challenge">Challenge</h4>

<p>Use the dataset here: <a href="https://snap.stanford.edu/data/loc-gowalla.html">https://snap.stanford.edu/data/loc-gowalla.html</a></p>

<ul>
  <li>Assume a “stalker” is someone who, in this dataset, visits some of the same locations as another person, after the other person goes to that location.</li>
  <li>A “stalker score” for a pair of people, A &amp; B, is the number of locations for which A has visited a location followed by B visiting that same location in the future.</li>
  <li>Any given location should be counted once in the score, so a stalker score can never be higher than the number of unique locations that A and B have in common.</li>
</ul>

<p>Use the datasets from the web page above to answer the following questions:</p>

<ol>
  <li>Which friend pair has the highest “stalker score”?</li>
  <li>Which non-friend pair has the highest “stalker score”?</li>
</ol>

<p>You can use any tools you want to solve this puzzle, except asking for help from other people. Please feel free to email at any time for any clarifications.</p>

<p>Please give the winning user id pairs and “stalker score” for each question, and please explain your solution methods, including any source code if you wrote any.</p>

<p>There are many libraries available that can make this challenge easier, so we are offering bonus points for:</p>
<ul>
  <li>Using as little RAM as possible</li>
  <li>Exection speed</li>
  <li>Pure python solution using base modules</li>
</ul>


# <b> <h3> <em>Script and Macine characteristics</em> </h3></b>

<p style="text-align:justify">
    This project will be implemented with PIP-8 style guide of python, I will evaluate computation time, size in memory, if it is possible, I will implement concurrency.The computer in which this scrip will be developed has a CPU intel core i5 of third generation with four threads and 2.5 Gz, 8GB of RAM, standard hard-disk of 512GB,  executing Ubuntu 16.04.

This script is powered by python 3.7 (anaconda distribution). 
</p>

# 1. Loading Datasets

In [190]:
import pandas as pd 

PATH_CHECKINGS = "datasets/Gowalla_totalCheckins.txt"
PATH_EDGES = "datasets/Gowalla_edges.txt"

In [191]:
data_edges = pd.read_csv(PATH_EDGES, sep="\t", header=None)

In [192]:
data_edges.head()

,0,1
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5


In [193]:
data_checkins = pd.read_csv(PATH_CHECKINGS, sep="\t", header=None)
data_checkins.columns = ["user", "check-in time", "latitude", "longitude", "location id"]
data_checkins.set_index("location id", inplace = True)

In [92]:
data_checkins.head(5)

,user,check-in time,latitude,longitude
location id,,,,
22847,0,2010-10-19T23:55:27Z,30.235909,-97.795140
420315,0,2010-10-18T22:17:43Z,30.269103,-97.749395
316637,0,2010-10-17T23:42:03Z,30.255731,-97.763386
16516,0,2010-10-17T19:26:05Z,30.263418,-97.757597
5535878,0,2010-10-16T18:50:42Z,30.274292,-97.740523


The plan to addres this problem wil be:

-  <b> <em> Preprocess the raw-data: </em></b></i> <p style="text-align:justify">let's deletete the information that there is not important for our challenge, (<b><em>latitude, longitude</em></b>). in each step it will shown how change the size of the data. I will create a Sparse matrix Object to represent the matrices. we will have 3 matrix: friend_relationship, stalker_score, locations_chekins. for better explanation, in each step there will be a little discription about what I'm doing.</p>


# 2. Preprocessing Data

## 2.1 Cleaning of  "latitude" and "longitude" 

In [194]:
import sys

def size_mb(x):
    """return the size in megabytes of one obeject"""
    
    bytes_size = sys.getsizeof(x)
    
    return (bytes_size /(1024*1024))


In [195]:
print("Size of checkins raw_dataset: %.2f MB" %  size_mb(data_checkins))

Size of checkins raw_dataset: 669.74 MB


In [196]:
print("Size of Friendship network raw_dataset: %.2f MB" %  size_mb(data_edges))

Size of Friendship network raw_dataset: 29.00 MB


In [96]:
#Elimination of latitude and longitude columns
data_checkins.drop(['latitude', 'longitude'], axis=1, inplace = True)

In [97]:
print("Size of checkins without latitude and longitude: %.2f MB" %  size_mb(data_checkins))

Size of checkins without latitude and longitude: 571.43 MB


## 2.2 Pre-processing check-in time
Now the date is type string, let's change it to python date type.

In [98]:
data_checkins['check-in time'] = pd.to_datetime(data_checkins['check-in time'], infer_datetime_format=True)

In [99]:
data_checkins.sort_values(by=['location id'], inplace = True)
data_checkins.head()

,user,check-in time
location id,,
8904,343,2009-08-21 03:01:19
8904,576,2009-10-05 20:19:24
8904,24,2009-02-05 06:27:43
8904,310,2009-12-06 22:51:49
8904,5164,2010-01-25 01:29:46


In [100]:
print("Size of checkins with date pre-processed: %.2f MB" %  size_mb(data_checkins))

Size of checkins with date pre-processed: 147.47 MB


<p style="text-align:justify"> We could see how just adjusting the datatype of the <b><em>"check-in time"</em></b> we achieve a huge reduction of the size of our principal dataset.

Let's check if there is some missing value
</p>

## 2.3 Checking missing values

In [101]:
data_checkins.isna().sum()

user             0
check-in time    0
dtype: int64

In [102]:
data_edges.isna().sum()

0    0
1    0
dtype: int64

<p> There is not missing values in both datasets, so we can continue.

I think we can reduce even more the datasets size.
</p>

## 2.4 sparse matrix

<p style="text-align:justify"> To solve our challenge we can implemnt sparce matrices, with help of dicts.</p> 
     

In [202]:
class SparceMatrix:
    """ 
    
    Object for manage big matrices with a 
    hugue amount of NaN or repteaded values.
    
    based in python dicts
    """
    
    def __init__(self):
        self.__matrix = {} #private attribute
        
    def row_exist(self, row_id):
        if not row_id in self.__matrix:
            return False
    
    def get_row(self,row_id):
        if row_exist(self, row_id):
            return self.__matrix[row_id]
        return False;
    
    def element_exist(self, row_id, col_id):
        try:
            if not row_id in self.__matrix:
                return False

            if col_id in self.get_row(row_id):
                return True
            return False
        except ValueError:
            print("you are using a unidimesional version, this version not has col_id") 
    
    def get_val(self, row_id, col_id):
        try:
            return self.__matrix[row_id][col_id]
        except ValueError:
            print("you are using a unidimesional version, this version not has col_id") 
        #return self.__matrix[row_id][col_id]
    
    def get_index(self):
        return self.__matrix.keys()
    
    def get_matrix(self):
        return self.__matrix
    
    def append(self, row_id, k, col_id = None):
        """
        add a new item.
        
        k could be any type of data.
        """
        if col_id is None:
            if  row_id in self.__matrix:
                self.__matrix[row_id].update(k)
            else:
                self.__matrix[row_id] = k
        else:
            if  row_id in self.__matrix:
                self.__matrix[row_id].update({col_id: k})
            else:
                self.__matrix[row_id] = {col_id: k}
       
    
    def update(self, row_id, k, col_id = None):
        """
        update item in the position [row_id, col_id].
        
        k could be any type of data.
        """
        
        if col_id is None:
            self.__matrix[row_id] = k
        else:
            self.__matrix[row_id][col_id] = k 
        
        
    def concat(self, Matrix):
        self.__matrix.update(Matrix.get_matrix())
    
    def pprint(self, k = 4):
        """ 
        print the rows of the matrix
        
        by defect print the first 5 
        """
        
        counter = 0
        for x, y in self.__matrix.items():
            print(f'{x} -> {y}')
            counter += 1
            if counter == k :
                break  

# 2.5 transforming representation of data

<p style="text-align:justify"> Now, that we have our class SparceMatrix we can represent our datasets: Checkins and friendship network, and our result set: stalker_classification, in this data structure. It definetly will reduce the size in RAM of our datasets and it will provide more performance in our <b><em>step 3. Processing.</em></b></p>

### 2.5.1 Transformation of data_checkins

<p> For this transformation let's make some asumptions:
    <ul>
        <li>based on the definition of the challenger: <em>"stalker is someone who, in this dataset, visits some of the same locations as another person, after the other person goes to that location."</em>  we could say that there is not timestap defined, in this way we could say that if the person <em>A</em> went to the place <em>X</em> the 01/01/2020 and the person <em>B</em> went to the same place the 05/01/2020 B will be considered has staleker of <em>A</em>. It means that while: $$A_{date}(X) < B_{date}(X) \implies (A,B) = 1 $$
this is valid for each place</li>
        <li>
            Using the previous assumption we could say that for our analysis we only need at most two dates for each user over one place: the first date that the user has visted the place and the last date, obviously there are users that has visited a certain place just one time, in this case we will use just one date. It will reduce the size of our dataset, because there are user that are made chekin more than two times in the same place, for example in their jobs. 
        </li>
        <br>
        <li>
            if A place x just has been visited by one user, for our challenge is useless, so let delete this places, it will reduce more the size of our pre-procesed dataset
        </li>
    </ul>
</p>

In [104]:
# we will use this to measure the time that takes execute a function
import time

In [18]:
#first create our sparse matrix 
locations_chekins = SparceMatrix()

In [106]:
#lets extract the unique ids of the places
location_ids = data_checkins.index.unique()
print(len(location_ids))

1280969


In [22]:
#lets calc the time:
# Start the stopwatch / counter 
start = time.time()
for location_id in location_ids:
    #select the users and dates
    location_checkins = data_checkins.loc[location_id]
    #ommiting the places with just one visitor
    if location_checkins.size > 2:
        #lets change the index for the users id
        location_checkins.reset_index(inplace = True)
        location_checkins.set_index("user", inplace = True)
        #lets extract the unique ids of the users that visited the place
        user_ids = location_checkins.index.unique()
        #ommit locations with just one visitor
        if len(user_ids) > 1:
            #lets extract the dates in wich each user has visited the place
            for user_id in user_ids:
                user_dates = location_checkins.loc[user_id]
                #more than one visit?
                # two because a df with just one register has a dimensionality of 2 
                if user_dates.size > 2:
                    #extract the first and last date
                    first_date = user_dates["check-in time"].min()
                    last_date = user_dates["check-in time"].max()
                    #lets add this register to our SparceMatrix
                    locations_chekins.append(location_id, user_id, [first_date,last_date])
                else:
                    date = user_dates["check-in time"]
                    #lets add this register to our SparceMatrix
                    locations_chekins.append(location_id, user_id, date)
        else:
            pass
    else:
        pass
            
#cleaning vars
del location_id, location_checkins, user_ids, user_id, user_dates, first_date, last_date, date

# Stop the stopwatch / counter 
stop = time.time() 
print("This is the time that have taked pre process the data_checkins")
print(stop - start)

This is the time that have taked pre process the data_checkins
2685.626620054245


In [154]:
import pickle

In [157]:
#load / save the pre-processed files
def load_obj(path):
    pickle_in = open(path,"rb")
    obj = pickle.load(pickle_in)
    return obj

def save_obj(obj, path):
    #save the preprocessed data
    pickle_out = open( path + ".pickle","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

In [158]:
import gc
gc.collect()

991

In [22]:
# save_obj(locations_chekins, "locations_chekins")
locations_chekins = load_obj("locations_chekins.pickle")

#### lets see the size of our dataset after the preprocessing

In [23]:
print("this is the size of our dataset already pre-processed: ")
print(size_mb(locations_chekins.get_matrix()))

this is the size of our dataset already pre-processed: 
40.00010681152344


<p> we could see that from our initial size of 700 MB we have reduce it until 40 MB it is a reduction of 94.29%. For other hand the pre process has take arround 43 minutes, let see if we could implement concurrency to reduce this time and get the same result</p>

#### 2.5.1.1 applying concurrency

In [165]:
import concurrent.futures
import math

In [166]:
# we will use this  function for the concurrency
def split_list(array, n_splits = 4):
    """ this function split a lsit in almost equal parts"""
    
    splits = []
    delta = math.floor((len(array)/n_splits))
    for i in range(n_splits):
        if i == n_splits - 1 :
            a = array[(i*delta): ]
            splits.append(a)
        else:
            a = array[(i*delta) : ((i+1)*delta)]
            splits.append(a)
            
    return splits

In [167]:
def location_preprocessing(loc_ids, data_checkins):
    """ 
    this function will be executed for the threads
    
    this function will extract our useful data 
    of a user that has visted a place 
    """
    
    temp = SparceMatrix()
    
    for location_id in loc_ids:
        #select the users and dates
        location_checkins = data_checkins.loc[location_id]
        #ommiting the places with just one visit
        if location_checkins.size > 2:
            #lets change the index for the users id
            location_checkins.reset_index(inplace = True)
            location_checkins.set_index("user", inplace = True)
            #lets extract the unique ids of the users that visited the place
            user_ids = location_checkins.index.unique()
            #ommit locations with just one visitor
            if len(user_ids) > 1:
                #lets extract the dates in wich each user has visited the place
                for user_id in user_ids:
                    user_dates = location_checkins.loc[user_id]
                    #more than one visit?
                    # two because a df with just one register has a dimensionality of 2 
                    if user_dates.size > 2:
                        #extract the first and last date
                        first_date = user_dates["check-in time"].min()
                        last_date = user_dates["check-in time"].max()
                        #lets add this register to our SparceMatrix
                        temp.append(location_id, user_id, [first_date,last_date])
                    else:
                        date = user_dates["check-in time"]
                        #lets add this register to our SparceMatrix
                        temp.append(location_id, user_id, date)
            else:
                pass
        else:
            pass

    #cleaning vars
    del location_id, location_checkins, user_ids, user_id, user_dates, first_date, last_date, date
    return temp
    
    

In [168]:
#lets create te sparce matrix for concurrency
concurrency_locations_chekins = SparceMatrix()

In [169]:
# Start the stopwatch / counter 
start = time.time()

with concurrent.futures.ProcessPoolExecutor() as executor:
    split_loc_ids = split_list(location_ids, 6)
    results = [executor.submit(location_preprocessing, split, data_checkins) for split in split_loc_ids]
    
    for f in concurrent.futures.as_completed(results):
        concurrency_locations_chekins.concat(f.result())
        
# Stop the stopwatch / counter 
stop = time.time() 
print("This is the time that have taked pre process concurrently the data_checkins")
print(stop - start)

This is the time that have taked pre process concurrently the data_checkins
1407.0616173744202


In [170]:
#save_obj(concurrency_locations_chekins, "output/concurrency_locations_chekins")
concurrency_locations_chekins = load_obj("output/concurrency_locations_chekins.pickle")

In [171]:
print("this is the size of our dataset already pre-processed: ")
print(size_mb(concurrency_locations_chekins.get_matrix()))

this is the size of our dataset already pre-processed: 
20.000099182128906


### We see that we could have the same result in almost te middle of the time, i think that whit a graphic card it could be even faster

### 2.5.2 transforming  data_edges
<p> Like the data_checkins this information will be stored in a sparse matrix. We could say that, for be a non directed graph,  if $u_0$ is friend of $u_1$ so $u_1$ is friend of $u_0$. $$ if (u_0,u_1) \implies (u_1,u_0)$$ In this way and for use less space and perform the search fastly we could sort ascendig the user_id, and store the friendship relation  between ($u_0$, $u_1$) in the user list that has the smallest id</p>

In [197]:
data_edges.set_index(0, inplace=True)
data_edges.sort_index(inplace=True)
data_edges.head()

,1
0,
0,1
0,2
0,3
0,4
0,5


In [198]:
print("Size of raw data_edges: %.2f MB" %  size_mb(data_edges))

Size of raw data_edges: 29.00 MB


In [199]:
#let see the numebr of unique users
users = data_edges.index.unique()

In [200]:
print(len(users))

196591


In [ ]:
def edges_preprocessing(user_ids, data_edges):
    
    for user_id in user_ids:
        

In [186]:
data_edges.loc[0]

,1
0,
0,1
0,2
0,3
0,4
0,5
0,6
0,7
0,8
0,9


In [ ]:
import random
aleatorios = [random.randint(0,1000) for _ in range(100)]

In [ ]:
data_checkins.index.unique()

In [143]:
k = data_checkins.loc[8904]
k.shape

(12, 2)

In [144]:
k.reset_index(inplace=True)
k

,location id,user,check-in time
0,8904,343,2009-08-21 03:01:19
1,8904,576,2009-10-05 20:19:24
2,8904,24,2009-02-05 06:27:43
3,8904,310,2009-12-06 22:51:49
4,8904,5164,2010-01-25 01:29:46
5,8904,256,2010-09-19 01:47:05
6,8904,343,2009-05-28 23:40:31
7,8904,392,2009-12-17 23:02:42
8,8904,256,2010-04-24 23:57:00
9,8904,343,2009-06-20 01:40:01


In [145]:
k.set_index("user", inplace = True)

In [146]:
len(k.index.unique())

7

In [100]:
h = k.loc[44458]
print(h["check-in time"])

user
44458   2010-07-25 18:13:48
44458   2010-04-20 17:56:37
Name: check-in time, dtype: datetime64[ns]


In [101]:
print(h.size)

4


In [78]:
h = k.loc[44458]
print(h["check-in time"].min())
print(h["check-in time"].max())

2010-04-20 17:56:37
2010-07-25 18:13:48


In [89]:
if not hsize:
    print("eureka")

IndexError: tuple index out of range

In [ ]:
data_checkins

In [ ]:
k = data_checkins.iloc[0]
for i in k:
    print(i)

In [ ]:
#let's transform our dataset
x = 4
k = data_checkins.iloc[:x]
print(k)

In [124]:
m = SparceMatrix()
m.append(1, 1, [1,10])
m.append(1, 2, [3,10])
m.append(1, 3, [11,19])
m.append(1, 4, [20,23])

m.append(2, 1, [4,15])
m.append(2, 2, [6,20])
m.append(2, 3, [25,26])
m.append(2, 4, [27,28])


In [ ]:
k = m.get_row(1)

In [ ]:
print(k)

In [ ]:
m.exist(123, 4)

In [ ]:
m2 = SparceMatrix()

In [ ]:
for user, dates in k.items():
    current_first = dates[0]
    current_last = dates[1]
    for next_user, next_dates in k.items():
        if user != next_user:
            next_first = next_dates[0]
            next_last = next_dates[1]
            
            # comparacion de doble via
            
            # si la primera date del current es menor a la ultima date de check-in del next
            # next es saltalker de  current
            # estrictamente menor porque  el reto dice explicitamente en el futuro.
            if current_first < next_last:
                if m2.exist(user, next_user):
                    score = m2.get_val(user, next_user) 
                    m2.update(user, next_user, score + 1)
                else:
                    m2.append(user, next_user, 1)
            # si la primera data the next user es menor a la ultima data de current user
            #current user es stalker the next
            if next_first < current_last:
                if m2.exist(next_user, user):
                    score = m2.get_val(next_user, user) 
                    m2.update(next_user, user, score + 1)
                else:
                    m2.append(next_user, user, 1)
                
            else:
                pass
        else:
            pass
        
        
    

In [ ]:
m2.pprint()

<p> After create our class SparseMatrix we will start to transform our datasets in 
     The matrixes that we'll implement are shown in the next images:

In [ ]:
b = { 
    1:[1212, 1414],
    2:[1313, 1515]
    }

In [ ]:
a = {
    123 : { 
        1:[1212, 1414],
        2:[1313, 1515]
    },
    234 : [{1:[1212, 1414]}, {2:[1313, 1515]}]
}

In [ ]:
a[123][1]

In [ ]:
import turicreate as tc

In [ ]:

checkins = (tc.SFrame.read_csv(
    path_checkins, delimiter='\t', header=False).rename({'X1': 'user_id',
                                                          'X2' : 'checkin_ts',
                                                          'X3': 'lat', 
                                                          'X4' : 'lon',
                                                          'X5': 'location_id'})
            [["user_id", "location_id", "checkin_ts"]] )

In [ ]:
chin_ps = ( checkins.join( checkins, on = 'location_id' )
                    .rename( {'checkin_ts'   : 'checkin_ts_ee',
                              'checkin_ts.1' : 'checkin_ts_er',
                              'user_id'      : 'stalkee' ,
                              'user_id.1'    : 'stalker' } ) )

pairs_filtered = chin_ps[ (chin_ps['checkin_ts_ee'] < chin_ps['checkin_ts_er']) &
                          (chin_ps['stalkee'] != chin_ps['stalker']) ]

In [ ]:
final_result = ( pairs_filtered[['stalkee', 'stalker', 'location_id']]
                    .unique()
                    .groupby( ['stalkee','stalker'] ,
                               {"location_count" : agg.COUNT })
                    .topk( 'location_count', k=5 )
                    .materialize() )

print( final_result ) 